In [1]:
import pipeline

from sklearn.metrics import *

import pandas as pd
import numpy as np
from pipeline import analyze_best_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# setup
df = pd.read_csv('data/final_merged_df.csv')

splits = pipeline.split_last_year(df, colname='year_evictions')
cleaned_splits = [pipeline.clean_split(split) for split in splits]
labeled_splits = [pipeline.label(split, lower_bound=14, drop_column=True)
                  for split in cleaned_splits]

In [3]:
# classifiers
train_df, test_df = labeled_splits[-1] # take last split

params = {'C': 0.1, 'penalty': 'l1'}
model = LogisticRegression()
label ='label'
k = 0.16

analyze_best_model.top_k_blocks_clf(model, params, train_df, test_df, label, k).head(10)

,GEOID,label,pred_scores
6215,421010273003,1,1.000000
6978,421010301002,1,1.000000
8315,421010346001,1,1.000000
8511,421010353021,1,0.999999
6103,421010268001,1,0.999999
5389,421010243002,1,0.999999
8938,421010373003,1,0.999998
5711,421010257001,1,0.999994
8980,421010377002,1,0.999982
3009,421010122031,1,0.999939


In [4]:
# classifiers
train_df, test_df = labeled_splits[-1]


params = {'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 1000}
scored_df = pipeline.run_one_clf(train_df, test_df, 'RF', params, col_blacklist=['GEOID', 'year_evictions'])
pipeline.plot_precision_recall_n(scored_df)

ValueError: Number of features of the model must match the input. Model n_features is 97 and input n_features is 98 

In [ ]:
analyze_best_model.feature_importance_clf(model, params, train_df, test_df, label)

In [ ]:
# regresssions
train_df = cleaned_splits[5][0]
test_df = cleaned_splits[5][1]

params = {'C': 0.1}
model = LinearSVR()
label ='evictions'
k = 0.16

analyze_best_model.top_k_blocks_reg(model, params, train_df, test_df, label, k).head(10)

In [ ]:
analyze_best_model.feature_importance_reg(model, params, train_df, test_df, label).head(10)